## Setting

In [1]:
import os

def maybe_mkdir_p(directory: str) -> None:
    os.makedirs(directory, exist_ok=True)

# maic_dir = '/mnt/backup/'
# base_dir = os.path.join(maic_dir, 'working')
base_dir = os.getcwd()
input_dir = '/mnt/NM/dataset/'
temp_dir = os.path.join(base_dir, 'models/temp/convert_data/')

# maybe_mkdir_p(base_dir)
maybe_mkdir_p(temp_dir)

# ! git clone https://github.com/keemsir/nnUNet.git

respository_dir = os.path.join(base_dir, 'models/module')
os.chdir(respository_dir)

! pip install -e .

os.chdir(base_dir)

Looking in indexes: http://ftp.daumkakao.com/pypi/simple
Obtaining file:///tf/temp_submission/models/module
  Attempting uninstall: nnunet
    Found existing installation: nnunet 1.7.0
    Can't uninstall 'nnunet'. No files were found to uninstall.
  Running setup.py develop for nnunet
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import shutil
import json
import numpy as np
import pandas as pd

from collections import OrderedDict
from scipy import special
import copy
import cv2
from tqdm import tqdm
import h5py

# must install
import pydicom
import nibabel as nib
import SimpleITK as sitk

In [6]:
task_name = 'Task55_PETCT'
convert_name = 'Task555_PETCT'

main_dir = os.path.join(base_dir, 'models/main/nnUNet/nnunet')
mainT_dir = os.path.join(temp_dir, 'nnUNet/nnunet')

rawbase_dir = os.path.join(mainT_dir, 'nnUNet_raw_data_base/')

pp_dir = os.path.join(mainT_dir, 'preprocessed')
tasks_dir = os.path.join(mainT_dir, 'Tasks')
task_dir = os.path.join(tasks_dir, task_name)

model_dir = os.path.join(main_dir, 'nnUNet_trained_models')
Prediction_dir = os.path.join(main_dir, 'nnUNet_Prediction_Results')
result_dir = os.path.join(Prediction_dir, convert_name)

staple_dir = os.path.join(Prediction_dir, 'staple')

# 1. Data preprocessing
maybe_mkdir_p(tasks_dir)
maybe_mkdir_p(temp_dir)

# 2. Directory
maybe_mkdir_p(main_dir)
maybe_mkdir_p(model_dir)
maybe_mkdir_p(pp_dir)

# 3. Directory
maybe_mkdir_p(result_dir)
maybe_mkdir_p(staple_dir)


#Environment Setting
os.environ['nnUNet_raw_data_base'] = rawbase_dir #os.path.join(mainT_dir, 'nnUNet_raw_data_base')
os.environ['nnUNet_preprocessed'] = pp_dir #os.path.join(mainT_dir, 'preprocessed')
os.environ['RESULTS_FOLDER'] = './models' #os.path.join(main_dir, 'nnUNet_trained_models')

print('Setting Completed!')

Setting Completed!


## Testdataset Preprocessing

In [7]:
size_dic = {}
def hdf2nifti(hdf_folder: str, save_folder: str):
    # hdf_folder : [train_dir, test_dir] hdf5 file path
    # save_folder : [imagesTr, imagesTs] Save Folder path
    maybe_mkdir_p(os.path.join(save_folder, 'imagesTr'))
    maybe_mkdir_p(os.path.join(save_folder, 'imagesTs'))
    maybe_mkdir_p(os.path.join(save_folder, 'labelsTr'))
    print('Creating "{}" Image & Label ..'.format(os.path.basename(os.path.normpath(save_folder))))
    hdf5_files = os.listdir(hdf_folder)


    for hdf5_file in hdf5_files:


        hdf5_path = os.path.join(hdf_folder, hdf5_file)

        # image
        f_i = h5py.File(hdf5_path, 'r')
        ctarr = np.asarray(f_i['CT'])
        petarr = np.asarray(f_i['PET'])
        sizearr = np.asarray(f_i['Size'])
        f_i.close()
        
        size_dic[hdf5_file[:17]] = sizearr # new

        SLICE_SIZE_X, SLICE_SIZE_Y, SLICE_COUNT = ctarr.shape
        images = np.empty([SLICE_SIZE_X, SLICE_SIZE_Y, SLICE_COUNT, 0], dtype=np.single)

        image_ct = np.expand_dims(ctarr, axis=3)
        images = np.append(images, image_ct, axis=3)
        image_pet = np.expand_dims(petarr, axis=3)
        images = np.append(images, image_pet, axis=3)

        niim = nib.Nifti1Image(images, affine=np.eye(4))
        nib.save(niim, os.path.join(save_folder, 'imagesTs/{}.nii.gz'.format(hdf5_file[:17])))


    print('"{}" Image & Label Completed !!'.format(os.path.basename(os.path.normpath(save_folder))))
    print('Image Patient : {}'.format(len(os.listdir(input_dir))))

hdf2nifti(input_dir, task_dir)

Creating "Task55_PETCT" Image & Label ..


FileNotFoundError: [Errno 2] No such file or directory: '/mnt/NM/dataset/'

In [19]:
def json_mk(save_dir: str):
    # Path
    imagesTr = os.path.join(save_dir, 'imagesTr')
    imagesTs = os.path.join(save_dir, 'imagesTs')
    maybe_mkdir_p(imagesTr)
    maybe_mkdir_p(imagesTs)

    overwrite_json_file = True
    json_file_exist = False

    if os.path.exists(os.path.join(save_dir, 'dataset.json')):
        print('dataset.json already exist!')
        json_file_exist = True

    if json_file_exist == False or overwrite_json_file:

        json_dict = OrderedDict()
        json_dict['name'] = "PETCT"
        json_dict['description'] = "Medical Image AI Challenge 2021"
        json_dict['tensorImageSize'] = "4D"
        json_dict['reference'] = "https://maic.or.kr/competitions/"
        json_dict['licence'] = "SNUH"
        json_dict['release'] = "18/10/2021"

        json_dict['modality'] = {
            "0": "CT",
            "1": "PET"
        }
        json_dict['labels'] = {
            "0": "background",
            "1": "Aorta"
        }

        train_ids = sorted(os.listdir(imagesTr))
        test_ids = sorted(os.listdir(imagesTs))
        json_dict['numTraining'] = len(train_ids)
        json_dict['numTest'] = len(test_ids)

        json_dict['training'] = [{'image': "./imagesTr/%s" % i, "label": "./labelsTr/%s" % i} for i in train_ids]

        json_dict['test'] = ["./imagesTs/%s" % i for i in test_ids] #(i[:i.find("_0000")])

        with open(os.path.join(save_dir, "dataset.json"), 'w') as f:
            json.dump(json_dict, f, indent=4, sort_keys=False)

        if os.path.exists(os.path.join(save_dir, 'dataset.json')):
            if json_file_exist == False:
                print('dataset.json created!')
            else:
                print('dataset.json overwritten!')
json_mk(task_dir)

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/temp/nnUNet/nnunet/Tasks/Task55_PETCT'

In [ ]:

!nnUNet_convert_decathlon_task -i ./models/temp/convert_data/nnUNet/nnunet/Tasks/Task55_PETCT -output_task_id 555 # -i : task_dir
# !nnUNet_plan_and_preprocess -t 555 # --verify_dataset_integrity


## File path

## Prediction

In [13]:
!nnUNet_predict -i ./models/temp/convert_data/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task555_PETCT/imagesTs -o ./models/result/2d -t 555 -tr nnUNetTrainerV2 -m 2d --disable_tta
!nnUNet_predict -i ./models/temp/convert_data/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task555_PETCT/imagesTs -o ./models/result/3d_fullres -t 555 -tr nnUNetTrainerV2 -m 3d_fullres --disable_tta



Please cite the following paper when using nnUNet:

Isensee, F., Jaeger, P.F., Kohl, S.A.A. et al. "nnU-Net: a self-configuring method for deep learning-based biomedical image segmentation." Nat Methods (2020). https://doi.org/10.1038/s41592-020-01008-z


If you have questions or suggestions, feel free to open an issue at https://github.com/MIC-DKFZ/nnUNet

using model stored in  ./model/nnUNet/2d/Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1
This model expects 2 input modalities for each image
Found 1 unique case ids, here are some examples: ['23010018_20141226']
If they don't look right, make sure to double check your filenames. They must end with _0000.nii.gz etc
number of cases: 1
number of cases that still need to be predicted: 1
emptying cuda cache
loading parameters for folds, None
folds is None so we will automatically look for output folders (not using 'all'!)
found the following folds:  ['./model/nnUNet/2d/Task555_PETCT/nnUNetTrainerV2__nnUNetPlansv2.1/fold_0', './model/nn

## Staple

In [ ]:
import numpy as np
from scipy import special
import os
import copy
import nibabel as nib
from typing import List
import pandas as pd


# Utils ..
def subdirs(folder: str, join: bool = True, prefix: str = None, suffix: str = None, sort: bool = True) -> List[str]:
    if join:
        l = os.path.join
    else:
        l = lambda x, y: y
    res = [l(folder, i) for i in os.listdir(folder) if os.path.isdir(os.path.join(folder, i))
           and (prefix is None or i.startswith(prefix))
           and (suffix is None or i.endswith(suffix))]
    if sort:
        res.sort()
    return res

def subfiles(folder: str, join: bool = True, prefix: str = None, suffix: str = None, sort: bool = True) -> List[str]:
    if join:
        l = os.path.join
    else:
        l = lambda x, y: y
    res = [l(folder, i) for i in os.listdir(folder) if os.path.isfile(os.path.join(folder, i))
           and (prefix is None or i.startswith(prefix))
           and (suffix is None or i.endswith(suffix))]
    if sort:
        res.sort()
    return res

def maybe_mkdir_p(directory: str) -> None:
    os.makedirs(directory, exist_ok=True)


def find3d_ind(bigMask_):
    indV = np.argwhere(bigMask_ == True)
    indVx = indV[:, 0]
    indVy = indV[:, 1]
    indVz = indV[:, 2]

    xMin = np.min(indVx)
    xMax = np.max(indVx)
    yMin = np.min(indVy)
    yMax = np.max(indVy)
    zMin = np.min(indVz)
    zMax = np.max(indVz)
    return xMin, xMax, yMin, yMax, zMin, zMax


def staple_wjcheon(D, iterlim, p, q):
    # ---- inputs:
    # *D: a matrix of N(voxels) x R(binary decisions by experts)
    # *p: intial sensitivity
    # *q: intial specificity
    # *iterlim: iteration limit
    # ---- outputs:
    # *p: final sensitivity estimate
    # *q: final specificity estimate
    # *W: estimated belief in true segmentation
    [N, R] = np.shape(D)
    Tol = 1e-4
    iter = 0
    gamma = np.sum(np.sum(D, axis=0) / (R * N))
    W = np.zeros((N, 1), dtype=np.single)
    S0 = np.sum(W)

    stapleV = []
    sen = []
    spec = []
    Sall = []
    while (True):
        iter = iter + 1
        Sall.append(S0)

        ind1 = np.equal(D, 1)
        ind0 = np.equal(D, 0)
        ind1_not = np.logical_not(ind1)
        ind0_not = np.logical_not(ind0)

        p = np.repeat(p, N, axis=0)
        p1 = copy.deepcopy(p)
        p0 = copy.deepcopy(1 - p1)

        p1[ind1_not] = 1
        p0[ind0_not] = 1
        a = gamma * np.multiply(np.prod(p1, axis=1), np.prod(p0, axis=1))
        del p1, p0

        q = np.repeat(q, N, axis=0)
        q0 = copy.deepcopy(q)
        q1 = copy.deepcopy(1 - q0)
        q1[ind1_not] = 1
        q0[ind0_not] = 1
        del ind1, ind0, ind1_not, ind0_not
        b = (1 - gamma) * np.multiply(np.prod(q0, axis=1), np.prod(q1, axis=1))
        del q1, q0

        W = np.divide(a, a + b)
        W = np.reshape(W, (1, len(W)))

        del a, b, p, q

        p = np.divide(np.matmul(W, D), np.sum(W))
        q = np.divide(np.matmul(1 - W, 1 - D), np.sum(1 - W))
        # Check convergence
        S = np.sum(W)
        if np.abs(S - S0) < Tol:
            print("STAPLE converged in {} iterations".format(iter))
            break
        else:
            S0 = S

        # Check iteration limit
        if (iter > iterlim):
            print("STAPLE: Number of iterations exceeded without convergence (convergence tolerance = %e)".format(Tol))
            break

    return W, p, q, Sall


def getUniformScanXYZVals_standardalone(rtstStruct):
    sizeArray = np.shape(rtstStruct)
    sizeDim1 = sizeArray[0] - 1  # sizeArray[0] - 1
    sizeDim2 = sizeArray[1] - 1  # sizeArray[1] - 1

    xOffset = 0
    yOffset = 0
    firstZValue = 0
    grid2Units = 4.6875  # 0.9765625
    grid1Units = 4.6875  # 0.9765625
    sliceThickness = 3.27001953

    # xVals = xOffset - (sizeDim2*grid2Units)/2 : grid2Units :
    xSt = xOffset - (sizeDim2 * grid2Units) / 2
    xEnd = xOffset + (sizeDim2 * grid2Units) / 2 + grid2Units
    xVals = np.arange(xSt, xEnd, grid2Units)

    ySt = yOffset - (sizeDim1 * grid1Units) / 2
    yEnd = yOffset + (sizeDim1 * grid1Units) / 2 + grid2Units
    yVals = np.arange(ySt, yEnd, grid1Units)
    yVals = np.flip(yVals)

    nZSlices = sizeArray[2];
    zSt = firstZValue
    zEnd = sliceThickness * (nZSlices - 1) + firstZValue + sliceThickness
    zVals = np.arange(zSt, zEnd, sliceThickness)

    return xVals, yVals, zVals


def kappa_stats(D, ncat):
    [N, M] = np.shape(D)
    lk = len(ncat)
    x = []
    for iterVal in range(0, lk):
        x.append(np.sum(np.equal(D, ncat[iterVal]), axis=1))
    x = np.transpose(x)

    p = np.divide(np.sum(x, axis=0), (N * M)) # Default : axis=0
    eps = np.finfo(float).eps
    k_a = np.sum(np.multiply(x, M - x), axis=0) # Default : axis=0
    k_b = (N * M * (M - 1)) * np.multiply(p, (1 - p)) + eps
    k = 1 - np.divide(k_a, k_b)
    sek = np.sqrt(2 / (N * M * (M - 1)))
    pk = drxlr_get_p_gaussian(np.divide(k, sek)) / 2
    kappa_a = N * M * M - np.sum(np.sum(np.multiply(x, x)))
    kappa_b = N * M * (M - 1) * np.sum(np.multiply(p, (1 - p))) + eps
    kappa = 1 - (kappa_a / kappa_b)
    sekappa_a = np.sum(np.multiply(p, (1 - p)) * np.sqrt(N * M * (M - 1)) + eps)
    sekappa_b = np.power(np.sqrt(np.sum(np.multiply(p, (1 - p)))), 2) - np.sum(
        np.multiply(np.multiply(p, 1 - p), (1 - 2 * p)))
    sekappa = np.sqrt(2) / sekappa_a * sekappa_b
    z = kappa / sekappa
    pval = drxlr_get_p_gaussian(z) / 2

    return kappa, pval, k, pk


def drxlr_get_p_gaussian(x):
    p = special.erfc(np.abs(x) / np.sqrt(2))

    return p


def calConsensus_standardalone(rtstStructs):
    keysDictionary = list(rtstStructs.keys())
    bigMask = rtstStructs.get(keysDictionary[0])
    dictionaryLength = len(rtstStructs)
    for iter1 in range(0, dictionaryLength):
        bigMask = np.logical_or(bigMask, rtstStructs.get(keysDictionary[iter1]))

    iMin, iMax, jMin, jMax, kMin, kMax = find3d_ind(bigMask);

    averageMask3M = np.zeros((iMax - iMin + 1, jMax - jMin + 1, kMax - kMin + 1), dtype=np.single)
    rateMat = []
    for iter1 in range(0, dictionaryLength):
        mask3M = rtstStructs.get(keysDictionary[iter1])
        mask3M_ROI = np.asanyarray(mask3M[iMin:iMax + 1, jMin: jMax + 1, kMin: kMax + 1])
        averageMask3M = averageMask3M + mask3M_ROI
        mask3M_ROI_flat = mask3M_ROI.flatten()
        rateMat.append(mask3M_ROI_flat)
    averageMask3M = averageMask3M / dictionaryLength
    rateMat = np.transpose(rateMat)
    scanNum = 1
    iterlim = 100
    senstart = 0.9999 * np.ones((1, dictionaryLength))
    specstart = 0.9999 * np.ones((1, dictionaryLength))
    [stapleV, sen, spec, Sall] = staple_wjcheon(rateMat, iterlim, np.single(senstart), np.single(specstart))

    mean_sen = np.mean(sen)
    std_sen = np.std(sen, ddof=1)
    mean_spec = np.mean(spec)
    std_spec = np.std(spec, ddof=1)

    [xUnifV, yUnifV, zUnifV] = getUniformScanXYZVals_standardalone(mask3M)
    vol = (xUnifV[2] - xUnifV[1]) * (yUnifV[1] - yUnifV[2]) * (zUnifV[2] - zUnifV[1])
    vol = vol * 0.001

    numBins = 20
    obsAgree = np.linspace(0.001, 1, numBins)
    rater_prob = np.mean(rateMat, axis=0)
    chance_prob = np.sqrt(np.multiply(rater_prob, (1 - rater_prob)))
    chance_prob = np.reshape(chance_prob, (1, np.shape(chance_prob)[0]))
    chance_prob_mat = np.repeat(chance_prob, np.shape(rateMat)[0], axis=0)
    reliabilityV = np.mean(np.divide((rateMat - chance_prob_mat), (1 - chance_prob_mat)), axis=1)
    del rater_prob, chance_prob, chance_prob_mat

    volV = []
    volStapleV = []
    volKappaV = []
    for iter10 in range(0, len(obsAgree)):
        updatedValue = np.sum((averageMask3M.flatten() >= obsAgree[iter10]) * vol)
        volV.append(updatedValue)
        updatedValue2 = np.sum((stapleV.flatten() >= obsAgree[iter10]) * vol)
        volStapleV.append(updatedValue2)
        updatedValue3 = np.sum((reliabilityV.flatten() >= obsAgree[iter10]) * vol)
        volKappaV.append(updatedValue3)

    # calculate overall kappa
    [kappa, pval, k, pk] = kappa_stats(rateMat, [0, 1])
    min_vol = np.min(np.sum(rateMat, axis=0)) * vol
    max_vol = np.max(np.sum(rateMat, axis=0)) * vol
    mean_vol = np.mean(np.sum(rateMat, axis=0)) * vol
    sd_vol = np.std(np.sum(rateMat, axis=0), ddof=1) * vol

    print('-------------------------------------------')
    print('Overall kappa: {0:1.8f}'.format(kappa))
    print('p-value: {0:1.8f}'.format(pval))
    print('Mean Sensitivity: {0:1.8f}'.format(mean_sen))
    print('Std. Sensitivity: {0:1.8f}'.format(std_sen))
    print('Mean Specificity: {0:1.8f}'.format(mean_spec))
    print('Std. Specificity: {0:1.8f}'.format(std_spec))
    print('Min. volume: {0:1.8f}'.format(min_vol))
    print('Max. volume: {0:1.8f}'.format(max_vol))
    print('Mean volume: {0:1.8f}'.format(mean_vol))
    print('Std. volume: {0:1.8f}'.format(sd_vol))
    print('Intersection volume: {0:1.8f}'.format(volV[-1]))
    print('Union volume: {0:1.8f}'.format(volV[1]))
    print('-------------------------------------------')

    len_x, len_y, len_z = np.shape(averageMask3M)
    stapleV_reshape = np.reshape(stapleV, (len_x, len_y, len_z))
    staple3M = np.zeros_like(bigMask, dtype=np.single)
    staple3M[iMin:iMax + 1, jMin: jMax + 1, kMin: kMax + 1] = stapleV_reshape
    #
    reliabilityV_reshape = np.reshape(reliabilityV, (len_x, len_y, len_z))
    reliability3M = np.zeros_like(bigMask, dtype=np.single)
    reliability3M[iMin:iMax + 1, jMin: jMax + 1, kMin: kMax + 1] = reliabilityV_reshape
    #
    apparent3M = np.zeros_like(bigMask, dtype=np.single)
    apparent3M[iMin:iMax + 1, jMin: jMax + 1, kMin: kMax + 1] = averageMask3M

    return apparent3M, staple3M, reliability3M


##### Parameter #####

# folderlist_temp = ['21-11-03_13:12:07-models', '21-11-01_13:53:00-models']

mainPath = './models/result/'
savePath = './models/staple/'

maybe_mkdir_p(savePath)

folderList = subdirs(mainPath, join=False) # os.listdir(mainPath)

result_list = subfiles(os.path.join(mainPath, folderList[0]), join=False, suffix='.nii.gz')

targetWeight = 0.6

print("STAPLE process is starting...")


# PatientKey = os.listdir(mainPath) # mainPath : Patient Key List


for iterPatient in result_list: # PatientKey

    dataPerPatient_aorta = {} # Model 별 Stack

    # PatientKey = PatientKey + 'hdf5'

    for ModelList in folderList:
        # print(ModelList)

        # patientStack_aorta = []

        patientStack_aorta = np.array(nib.load(os.path.join('./models/result/{}'.format(ModelList), iterPatient)).dataobj)

        dataPerPatient_aorta[ModelList] = patientStack_aorta

    # STAPLE
    [apparent3M_label2, staple3M_label2, reliability3M_label2] = calConsensus_standardalone(dataPerPatient_aorta)
    mask2 = np.uint8((staple3M_label2 >= targetWeight))

    nii_stp = nib.Nifti1Image(mask2, affine=np.eye(4))
    nib.save(nii_stp, os.path.join(savePath, iterPatient))

print("STAPLE process is done...")




## Submission

In [ ]:
def get_suv_params(ptarr, roi):
    roi = np.asarray(roi>0, dtype=np.float)
    suvmax = np.max(ptarr*roi)
    suvmean = np.sum(ptarr*roi)/np.sum(roi)
    return suvmax, suvmean

def get_vol_params(ptzoom, roi):
    roi = np.asarray(roi>0, dtype=np.float)
    return np.prod(ptzoom) * np.sum(roi)

In [ ]:
PN_list = os.listdir('./models/staple/')
PN_list.sort()

for pn_l in PN_list:
    _, ext = os.path.splitext(pn_l)
    if ext == '.gz':
        
        pn = pn_l[:-7]
        PATIENT_NUM = pn

        ptarr = np.array(nib.load('./models/temp/convert_data/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task555_PETCT/imagesTs/{}_0001.nii.gz'.format(pn)).dataobj)
        ctarr = np.array(nib.load('./models/temp/convert_data/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task555_PETCT/imagesTs/{}_0000.nii.gz'.format(pn)).dataobj)
        pred_arr = np.array(nib.load('./models/staple/{}.nii.gz'.format(pn)).dataobj)

        # size = ([4.07283, 4.07283, 3.])
        size = ([4.6875, 4.6875, 3.27001953])
        # size = size_dic[PATIENT_NUM]
        
        #Calculate Mean SUV and Max SUV

        suvmax, suvmean = get_suv_params(ptarr, pred_arr)

        #Calculate Volume

        aorvol = get_vol_params(size, pred_arr)
        
        PATIENT_NUM = pn

        data = {'case' : [PATIENT_NUM], 'PD_Aorta_volume' : [aorvol], 'PD_SUVmean' : [suvmean]}
        df = pd.DataFrame(data)

        # .to_csv 
        if not os.path.exists('submission.csv'):
            df.to_csv('submission.csv', index=False, mode='w')
        else:
            df.to_csv('submission.csv', index=False, mode='a', header=False)
